In [2]:
# Code to read all RFB files in a folder, get termperature data from each RFB file and plot using plotly. Go through each subfolder in the folder
# Compute average, max and min temperature from each RFB file and plot using plotly
# For each SN, save average, max and min temperature for each run- 1,2,3
# Input : folder path containing RFB files
# Output : Interactive plotly box plot with 3 subplots for average, max and min.


from ast import Import
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import pandas as pd


def find_rfb_files(root_dir):
    # This function is to find all RFB files in the folder and any subfolders
    matching_files = []
    print(f"Searching for RFB files in directory: {root_dir}")
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".rfb") or filename.endswith(".RFB"):
                    full_path = os.path.join(dirpath, filename)
                    matching_files.append(full_path)
    return matching_files
# ********************  END Function  *************************

def read_rfb(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Find the line where data starts
    data_start_index = 0
    for i, line in enumerate(lines):
        if "[Raw Data]" in line:
            data_start_index = i + 1
            break
    
    # Read the data lines into a DataFrame
    data_lines = lines[data_start_index:]
    data = []
    for line in data_lines:
        parts = line.strip().split()
        #print(parts)
        if len(parts) == 7:
            try:
                line_number = int(parts[0])
                time_temp = parts[1]
                # Convert time to a suitable date time format if needed
                time = pd.to_datetime(time_temp, format='%H:%M:%S.%f')

                tempC = float(parts[2])
                volt = float(parts[3])
                wt = float(parts[4])
                pow = float(parts[5])
                state = int(parts[6])
                data.append([line_number, time, tempC, volt, wt, pow, state])
            except ValueError:
                continue  # Skip lines that can't be parsed
    
    df = pd.DataFrame(data, columns=['Line Number', 'Time', 'Temperature', 'Voltage', 'Weight', 'Power', 'State'])
    #df["Time"]= df["Time"]-min(df["Time"])
    return df
# ********************  END Function  *************************

def plot_rfb_data(folder_path):

    #Find all RFB files in the folder and any subfolders
    files = find_rfb_files(folder_path)
    if not files:
        print(f"No RFB files found in folder '{folder_path}'.")
        return

    data = {}
    # Get average, max and min temperature from each RFB file
    for file in files:
        fname = os.path.basename(file)
        SN = fname.split('-')[0]
        run_no = fname.split('-')[1].split('.')[0]

        df = read_rfb(file)
        avg_temp = df["Temperature"].mean()
        max_temp = df["Temperature"].max()
        min_temp = df["Temperature"].min()

        if run_no not in data:
            data[run_no] = {"SN": [], "Avg Temp": [], "Max Temp": [], "Min Temp": []}

        data[run_no]["SN"].append(SN)
        data[run_no]["Avg Temp"].append(avg_temp)
        data[run_no]["Max Temp"].append(max_temp)
        data[run_no]["Min Temp"].append(min_temp)

    # Create subplots
    fig = make_subplots(rows=3, cols=1, subplot_titles=("Average Temperature", "Maximum Temperature", "Minimum Temperature"))

    # Add box plots for each run number and overlay scatter plot of individual data points
    for run_no, values in data.items():
        fig.add_trace(go.Box(y=values["Avg Temp"], name=f'Run {run_no}'), row=1, col=1)
        fig.add_trace(go.Box(y=values["Max Temp"], name=f'Run {run_no}'), row=2, col=1)
        fig.add_trace(go.Box(y=values["Min Temp"], name=f'Run {run_no}'), row=3, col=1)
    fig.update_layout(height=900, width=700, title_text="RFB Temperature Data")
    fig.show()

# Example usage
data_folder_path = r"C:\work_folder\Efficiency\test_data_RFB"
plot_rfb_data(data_folder_path)

Searching for RFB files in directory: C:\work_folder\Efficiency\test_data_RFB
